In [28]:
from os.path import join as pjoin
import os

import matplotlib.pyplot as plt
import cv2

from torch.utils.tensorboard import SummaryWriter

from additonFunc import create_image_plot 

In [29]:
TBpath_orign_path = "TB_cache/new/MyUnet_FixedSet_augmented_5"
TBpath = TBpath_orign_path + "_valid_samples"

TBwriter = SummaryWriter(TBpath)

target_dir = pjoin(TBpath_orign_path, 'valid_samples')

folders = [pjoin(target_dir, filename) for filename in sorted(os.listdir(target_dir))]

def extract_folder_id(folds):
    max_id = -1
    lastfld = None
    for folder in folds:
        id = int(folder.split('_')[-1])
        if id > max_id:
            lastfld = folder
            max_id = id
    return lastfld
last_folder = extract_folder_id(folders)

images = [pjoin(last_folder, filename) for filename in sorted(os.listdir(last_folder)) if ".png" in filename]

for ind, image_path in enumerate(images):
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    TBwriter.add_figure('valid_test', create_image_plot(valid_test=image), ind)

In [22]:
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2

class CFG:
    # ============== comp exp name =============
    comp_name = 'vesuvius'

    # comp_dir_path = './'
    comp_dir_path = ''
    comp_folder_name = ''
    # comp_dataset_path = f'{comp_dir_path}datasets/{comp_folder_name}/'
#     comp_dataset_path = f'{comp_dir_path}{comp_folder_name}/'
    comp_dataset_path = ''
    
    exp_name = 'vesuvius_2d_slide_exp004'

    # ============== pred target =============
    target_size = 1

    # ============== model cfg =============
    model_name = 'FPN'
    backbone = 'efficientnet-b3'
    # backbone = 'se_resnext50_32x4d'

    in_chans = 12 # 65
    # ============== training cfg =============
    size = 256
    tile_size = 256
    stride = tile_size // 2

    train_batch_size = 32 # 32
    valid_batch_size = train_batch_size * 2
    use_amp = True

    scheduler = 'GradualWarmupSchedulerV2'
    # scheduler = 'CosineAnnealingLR'
    epochs = 30 # 30

    # adamW warmupあり
    warmup_factor = 10
    # lr = 1e-4 / warmup_factor
    lr = 1e-4 / warmup_factor

    # ============== fold =============
    valid_id = 1

    # objective_cv = 'binary'  # 'binary', 'multiclass', 'regression'
    metric_direction = 'maximize'  # maximize, 'minimize'
    # metrics = 'dice_coef'

    # ============== fixed =============
    pretrained = True
    inf_weight = 'best'  # 'best'

    min_lr = 1e-6
    weight_decay = 1e-6
    max_grad_norm = 1000

    print_freq = 50
    num_workers = 4

    seed = 42

    # ============== set dataset path =============
    print('set dataset path')

    outputs_path = f'outs'

    submission_dir = outputs_path + 'submissions/'
    submission_path = submission_dir + f'submission_{exp_name}.csv'

    model_dir = outputs_path + \
        f'{comp_name}-models/'

    figures_dir = outputs_path + 'figures/'

    log_dir = outputs_path + 'logs/'
    log_path = log_dir + f'{exp_name}.txt'

    # ============== augmentation =============
    train_aug_list = [
        # A.RandomResizedCrop(
        #     size, size, scale=(0.85, 1.0)),
#         A.Resize(size, size),
        A.RandomCrop(size, size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
#         A.RandomBrightnessContrast(p=0.75),
        A.ShiftScaleRotate(p=0.75),
        A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(),
                A.MotionBlur(),
                ], p=0.4),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        A.CoarseDropout(max_holes=1, max_width=int(size * 0.3), max_height=int(size * 0.3), 
                        mask_fill_value=0, p=0.5),
        # A.Cutout(max_h_size=int(size * 0.6),
        #          max_w_size=int(size * 0.6), num_holes=1, p=1.0),
        A.Normalize(
            mean= [0] * in_chans,
            std= [1] * in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]

    valid_aug_list = [
        A.Resize(size, size),
        A.Normalize(
            mean= [0] * in_chans,
            std= [1] * in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]
    
    def __create_attr_info(self, atr_name):
        atr_info = self.__getattribute__(atr_name)
        try:
            atr_info_str = str(atr_info)
        except:
            atr_info_str = "NSC"
        return f"{atr_name}: '{atr_info_str}'"

    def __str__(self):
        result_string = ""
        attrs_list = [attr for attr in dir(self) if not attr.startswith('__') and not attr.startswith("_CFG__")]
        for attr in attrs_list:
            result_string += self.__create_attr_info(attr) + '\n'
        return result_string


set dataset path


In [24]:
print(str(CFG))

<class '__main__.CFG'>
